In [2]:
import os
import sys
from pathlib import Path

CURRENT_DIRECTORY = Path(os.getcwd())
ROOT_DIRECTORY = (CURRENT_DIRECTORY / "..").absolute().resolve()

print(f"Current directory: {CURRENT_DIRECTORY}")
print(f"Root directory: {ROOT_DIRECTORY}")

sys.path.append(str(ROOT_DIRECTORY))

Current directory: /home/ubuntu/arga-arc/tf_coder
Root directory: /home/ubuntu/arga-arc


In [5]:
import typing as t
import json
from pprint import pprint
from dataclasses import dataclass
import tensorflow as tf
import numpy as np
import math
from config import CONFIG
from openai import OpenAI
import re
from collections import Counter
import random
import ast

pprint(CONFIG.__dict__.keys())

OPENAI = OpenAI(api_key=CONFIG.OPENAI_SECRET_KEY, organization=CONFIG.OPENAI_ORGANIZATION)

dict_keys(['OPENAI_SECRET_KEY', 'OPENAI_ORGANIZATION'])


In [7]:
OUTPUTS = CURRENT_DIRECTORY / "tfcoder_dataset_with_completions.in_context_ordered_by_weight.json"
OUTPUTS_JSON = json.loads(OUTPUTS.read_text())

pprint(OUTPUTS_JSON[0])

{'completions': ['return tf.stack([tf.math.bincount(in1), in1], axis=1)',
                 'return tf.stack([tf.divide(in1, tf.shape(in1)[0]), '
                 'tf.mod(in1, tf.shape(in1)[0])], axis=1)',
                 'return tf.stack([tf.math.floor_div(in1, tf.constant(2, '
                 'dtype=tf.int32)), tf.math.floormod(in1, tf.constant(2, '
                 'dtype=tf.int32))], axis=1)',
                 'return tf.stack([tf.floor_div(in1, 2), tf.mod(in1, 2)], '
                 'axis=1)',
                 'return tf.stack([tf.math.floor_divide(in1, '
                 'tf.shape(in1)[0]), tf.math.mod(in1, tf.shape(in1)[0])], '
                 'axis=1)',
                 'return tf.stack([tf.math.floor_divide(in1, '
                 'tf.reduce_max(in1) + 1), tf.math.floormod(in1, '
                 'tf.reduce_max(in1) + 1)], axis=1)',
                 'return tf.stack([tf.math.floor_div(in1, tf.reduce_max(in1) + '
                 '1), tf.math.floormod(in1, tf.reduce_max(in1)

In [76]:
COMPLETIONSES = [
    output['response']['completions'] if 'response' in output else None 
    for output in OUTPUTS_JSON
]

print(len(COMPLETIONSES))
pprint(COMPLETIONSES[0])

72
['return tf.stack([tf.math.bincount(in1), in1], axis=1)',
 'return tf.stack([tf.divide(in1, tf.shape(in1)[0]), tf.mod(in1, '
 'tf.shape(in1)[0])], axis=1)',
 'return tf.stack([tf.math.floor_div(in1, tf.constant(2, dtype=tf.int32)), '
 'tf.math.floormod(in1, tf.constant(2, dtype=tf.int32))], axis=1)',
 'return tf.stack([tf.floor_div(in1, 2), tf.mod(in1, 2)], axis=1)',
 'return tf.stack([tf.math.floor_divide(in1, tf.shape(in1)[0]), '
 'tf.math.mod(in1, tf.shape(in1)[0])], axis=1)',
 'return tf.stack([tf.math.floor_divide(in1, tf.reduce_max(in1) + 1), '
 'tf.math.floormod(in1, tf.reduce_max(in1) + 1)], axis=1)',
 'return tf.stack([tf.math.floor_div(in1, tf.reduce_max(in1) + 1), '
 'tf.math.floormod(in1, tf.reduce_max(in1) + 1)], axis=1)',
 'return tf.stack([tf.math.floor_div(in1, tf.reduce_max(in1) + 1), '
 'tf.math.floormod(in1, tf.reduce_max(in1) + 1)], axis=1)',
 'return tf.stack([tf.math.bincount(in1), tf.cast(in1, tf.int32)], axis=1)',
 'return tf.stack([tf.divide(in1, tf.shape(in

In [112]:
TEST_COMPLETIONS = [
    COMPLETIONSES[0][0],
    "return tf.pad(in1, [[0,0],[0,1]])",
    "return tf.gather(in1, in2, batch_dims=1)",
    "indices = tf.stack([tf.range(tf.shape(in2)[0]), in2], axis=1)\nreturn tf.gather_nd(in1, indices)",
    "return tf.cast(tf.sequence_mask(in1, maxlen=tf.reduce_max(in1)), tf.int32)",
    "return tf.where(in1, in2, tf.multiply(in2, tf.constant(-10)))",
    "return tf.tensor_scatter_nd_update(in1['tensor'], in1['indices'], in1['updates'])",
    "return tf.reduce_any(tf.one_hot(in1, 9), axis=1, keepdims=False)",
]
TEST_ASTS = [ast.parse(completion) for completion in TEST_COMPLETIONS]

In [113]:
def normalize_completion(completion: str) -> str:
    lines = completion.split("\n")
    normalized_lines = []
    for line in lines:
        normalized_line = re.sub(r"([\s]+)return", "return", line)
        normalized_line = re.sub(r"([\s]+)([a-zA-Z_][a-zA-Z0-9_]*)([\s]+)=", r"\2 =", normalized_line)
        normalized_lines.append(normalized_line)
    normalized_completion = "\n".join(normalized_lines)
    return normalized_completion

In [117]:
class ConstantVisitor(ast.NodeVisitor):
    def __init__(self):
        self.constants = []
    
    def visit_Constant(self, node: ast.Constant):
        if type(node.value) == int:
            self.constants.append(node.value)

    def visit_UnaryOp(self, node: ast.UnaryOp):
        if isinstance(node.op, ast.USub) and isinstance(node.operand, ast.Constant):
            self.constants.append(-node.operand.value)
        else:
            self.generic_visit(node)

def get_constants(ast_node: ast.Module) -> t.List[str]:
    visitor = ConstantVisitor()
    visitor.visit(ast_node)
    return visitor.constants

for completion, ast_ in zip(TEST_COMPLETIONS, TEST_ASTS):
    print(completion)
    print(get_constants(ast_))
    print()


return tf.stack([tf.math.bincount(in1), in1], axis=1)
[1]

return tf.pad(in1, [[0,0],[0,1]])
[0, 0, 0, 1]

return tf.gather(in1, in2, batch_dims=1)
[1]

indices = tf.stack([tf.range(tf.shape(in2)[0]), in2], axis=1)
return tf.gather_nd(in1, indices)
[0, 1]

return tf.cast(tf.sequence_mask(in1, maxlen=tf.reduce_max(in1)), tf.int32)
[]

return tf.where(in1, in2, tf.multiply(in2, tf.constant(-10)))
[-10]

return tf.tensor_scatter_nd_update(in1['tensor'], in1['indices'], in1['updates'])
[]

return tf.reduce_any(tf.one_hot(in1, 9), axis=1, keepdims=False)
[9, 1]



In [118]:
ASTSES = []

for completions in COMPLETIONSES:
    if completions is None:
        ASTSES.append(None)
        continue
    
    asts = []
    for completion in completions:
        try:
            asts.append(ast.parse(completion))
            continue
        except SyntaxError as e:
            pass

        normalized_completion = normalize_completion(completion) 
        try:
            asts.append(ast.parse(normalized_completion))
        except SyntaxError as e:
            print("# completion")
            print(completion)
            print()
            print("# normalized_completion")
            print(normalized_completion)
            print()
            print("# error")
            print(e)
            print()
            print()
            asts.append(None) 
        
    ASTSES.append(asts)

print(len(ASTSES))
pprint(ASTSES[0])

# completion
return tf.where(in1, in2, tf.multiply(-10, tf.range(tf.shape(in2)[0]));

# normalized_completion
return tf.where(in1, in2, tf.multiply(-10, tf.range(tf.shape(in2)[0]));

# error
invalid syntax (<unknown>, line 1)


72


In [119]:
CONSTANTSES = [
    [
        get_constants(ast_node) if ast_node is not None else None 
        for ast_node in ast_nodes
    ] if ast_nodes is not None else None
    for ast_nodes in ASTSES 
]

print(len(CONSTANTSES))
pprint(CONSTANTSES[0])

72
[[1],
 [0, 0, 1],
 [2, 2, 1],
 [2, 2, 1],
 [0, 0, 1],
 [1, 1, 1],
 [1, 1, 1],
 [1, 1, 1],
 [1],
 [0, 1]]


In [120]:
for output, constants in zip(OUTPUTS_JSON, CONSTANTSES):
    if 'response' not in output:
        continue
    output['response']['constants'] = constants
    output['response']['all_constants'] = list(set([
        constant 
        for constants in constants if constants is not None 
        for constant in constants
    ]))

pprint(OUTPUTS_JSON[0])

{'completions': ['return tf.stack([tf.math.bincount(in1), in1], axis=1)',
                 'return tf.stack([tf.divide(in1, tf.shape(in1)[0]), '
                 'tf.mod(in1, tf.shape(in1)[0])], axis=1)',
                 'return tf.stack([tf.math.floor_div(in1, tf.constant(2, '
                 'dtype=tf.int32)), tf.math.floormod(in1, tf.constant(2, '
                 'dtype=tf.int32))], axis=1)',
                 'return tf.stack([tf.floor_div(in1, 2), tf.mod(in1, 2)], '
                 'axis=1)',
                 'return tf.stack([tf.math.floor_divide(in1, '
                 'tf.shape(in1)[0]), tf.math.mod(in1, tf.shape(in1)[0])], '
                 'axis=1)',
                 'return tf.stack([tf.math.floor_divide(in1, '
                 'tf.reduce_max(in1) + 1), tf.math.floormod(in1, '
                 'tf.reduce_max(in1) + 1)], axis=1)',
                 'return tf.stack([tf.math.floor_div(in1, tf.reduce_max(in1) + '
                 '1), tf.math.floormod(in1, tf.reduce_max(in1)

In [121]:
OUTPUT_FILE = CURRENT_DIRECTORY / "tfcoder_dataset_with_completions_and_constants.json"
OUTPUT_FILE.write_text(json.dumps(OUTPUTS_JSON, indent=4))

259704